In [1]:
from pathlib import Path
import pandas as pd
import sys
import os
from helpers import process_position
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

three_hourly_file_list = list(Path("data/raw/3HOURLY_DATA").rglob("*.[cC][sS][vV]"))
level_one_file_list = list(Path("data/raw/L1_DATA").rglob("*.[cC][sS][vV]"))

In [2]:
#%%sh
#./reset.sh

In [3]:
overwrite = True
show_progress = True

for f in level_one_file_list[0:1]:
    if "2023" not in str(f):
        minute_label = "Minute"
        second_label = "Second"
    else:
        minute_label = "Min"
        second_label = "Sec"

    l1_year = pd.read_csv(f, usecols = ["BuoyID", "Lat", "Lon", "Year", "Month", "Day", "Hour", minute_label, second_label])
    l1_year = l1_year.rename(columns = {minute_label:"Minute", second_label:"Second", "Lat": "latitude", "Lon": "longitude"})
    l1_year["timestamp"] = pd.to_datetime(l1_year[["Year", "Month", "Day", "Hour", "Minute", "Second"]])
    l1_year = l1_year.set_index("timestamp")

    l1_year = l1_year.drop(columns=["Year", "Month", "Day", "Hour", "Minute", "Second"])

    groups = l1_year.groupby("BuoyID")
    total_buoys = len(groups)
    buoys_complete = 0

    for buoy, data in groups:
        if int(buoy) < 10**6:
            continue
        if show_progress:
            buoys_complete += 1
            sys.stdout.write('\r')
            count = buoys_complete / total_buoys
            sys.stdout.write("[%-100s] %d%%" % ('='*int(100*count), int(100/total_buoys*buoys_complete)))
            sys.stdout.flush()

        if not overwrite and f"{buoy}.csv" in os.listdir("data/clean_positions"):
           continue

        clean = process_position.clean_location_df(data)

        if clean is not None:
           clean.to_csv(f"data/clean_positions/{buoy}.csv")
    print(f)

[==================================================================                                  ] 66%

KeyboardInterrupt: 

In [4]:
# for f in three_hourly_file_list:
#     h3_data = pd.read_csv(f, usecols = ["BuoyID", "Lat", "Lon", "Year", "Hour", "DOY"])
#     for buoy, data in h3_data.groupby("BuoyID"):
#         data.to_csv(f"data/clean_positions/{buoy}.csv")

In [5]:
loc = "data/clean_positions/"
ranges = {}

for fname in os.listdir(loc):
    data = pd.read_csv(loc+fname)
    buoy_id = fname.split(".csv")[0]
    ranges[buoy_id] = []
    ranges[buoy_id].append(data.iloc[0]["datetime"])
    ranges[buoy_id].append(data.iloc[-1]["datetime"])

In [35]:
metadata = pd.DataFrame(ranges).transpose()
metadata = metadata.rename(columns={0: "start", 1: "end"})
metadata["start"] = pd.to_datetime(metadata["start"])
metadata["end"] = pd.to_datetime(metadata["end"])

starts = zip(metadata["start"], metadata.index)
ends = zip(metadata["end"], metadata.index)


(Timestamp('2020-12-31 23:00:00'), '300234066219740')
